# Чат бот инвестиций

### Проект выполнили:
* Николай Обливальный
* Name
* Алексей Бевз

#### Установка библиотеки для работы с ботом через API телеграмма

In [ ]:
#pip install pyTelegramBotAPI

Телеграмм умеет сообщать боту о действиях пользователя двумя способами: через ответ на запрос сервера (Long Poll), и через Webhook, когда сервер Телеграмма сам присылает сообщение о том, что кто-то написал боту. Второй способ явно выглядит лучше, но требует выделенного IP-адреса, и установленного SSL на сервере. Реализовать его наиболее простым способом можно опубликовав код на heroku.

В данном проекте используется Long Poll.

С помощью специального бота "BotFather" создали собственного бота и получили токен для работы с API.<br/>
UserName: InvestmentSampleBot
Название: Чат бот инвестиций

In [2]:
import telebot;
from telebot import types

In [3]:
token = ''

In [4]:
bot = telebot.TeleBot(token);

#### Домен

In [5]:
class InputParms():
    start_strategy = ''
    end_strategy = ''
    period_calculation = ''
    period_holding = ''
    
    def set_start_strategy(self, start_strategy):
        self.start_strategy = start_strategy
        
    def set_end_strategy(self, end_strategy):
        self.end_strategy = end_strategy
        
    def set_period_calculation(self, period_calculation):
        self.period_calculation = period_calculation
        
    def set_period_holding(self, period_holding):
        self.period_holding = period_holding
        
    def get_log(self):
        print(f'дата начала проверки стратегии: {self.start_strategy}')
        print(f'дата окончания проверки стратегии: {self.end_strategy}')
        print(f'период данных для расчета: {self.period_calculation}')
        print(f'период данных для удержания позиции: {self.period_holding}')
        
    def get_string(self):
        parms = []
        parms.append('Текущие значения параметров:')
        parms.append(f'дата начала проверки стратегии: {self.start_strategy}')
        parms.append(f'дата окончания проверки стратегии: {self.end_strategy}')
        parms.append(f'период данных для расчета: {self.period_calculation}')
        parms.append(f'период данных для удержания позиции: {self.period_holding}')
        return '\n'.join(parms)
    
    def is_exists_all_parms(self):
        return (self.start_strategy != '' and self.end_strategy != '' and self.period_calculation != '' and self.period_holding != '')
        

input_parms = InputParms()#### Функции для выполнения команд

In [6]:
input_parms = InputParms()

In [7]:
def startCmd(message):
    bot.send_message(message.from_user.id, "Привет! Это чат бот инвестиций.")
    analyzeCmd(message)

In [8]:
def helpCmd(message):
    bot.send_message(message.from_user.id, "Чем я могу тебе помочь?")

In [9]:
def getParamsCmd(message):
    bot.send_message(message.from_user.id, input_parms.get_string())

In [10]:
def analyzeCmd(message):
    bot.send_message(message.from_user.id, "Начинаем процесс ввода входных параметров для анализа.")
    bot.send_message(message.from_user.id, "Дата начала проверки стратегии:");
    bot.register_next_step_handler(message, get_start_strategy)

def get_start_strategy(message):
    input_parms.set_start_strategy(message.text)
    bot.send_message(message.from_user.id, "Дата окончания проверки стратегии:");
    bot.register_next_step_handler(message, get_end_strategy)

def get_end_strategy(message):
    input_parms.set_end_strategy(message.text)
    bot.send_message(message.from_user.id, "Период данных для расчета:");
    bot.register_next_step_handler(message, get_period_calculation)
    
def get_period_calculation(message):
    input_parms.set_period_calculation(message.text)
    bot.send_message(message.from_user.id, "Период данных для удержания позиции:");
    bot.register_next_step_handler(message, get_period_holding)
    
def get_period_holding(message):
    input_parms.set_period_holding(message.text)
    bot.send_message(message.from_user.id, "Ввод параметров завершен.");
    
    keyboard = types.InlineKeyboardMarkup();
    key_yes  = types.InlineKeyboardButton(text='Да', callback_data='yes');
    keyboard.add(key_yes);
    key_no   = types.InlineKeyboardButton(text='Нет', callback_data='no');
    keyboard.add(key_no);
    bot.send_message(message.from_user.id, text='Выполнить анализ?', reply_markup=keyboard)
    
@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if call.data == "yes": #call.data это callback_data, которая указана при объявлении кнопки
        analyze(call.message)
    else:
        bot.send_message(call.message.chat.id, 'Хорошо, значит в другой раз')

In [11]:
def analyze(call_message):
    if (not input_parms.is_exists_all_parms()):
        bot.send_message(call_message.chat.id, 'Введены не все входные параметры.')
        return
        
    #do something and return plots
    
    bot.send_message(call_message.chat.id, 'Анализ завершен')

In [12]:
botCmds = {
    "/start": startCmd,
    "/help": helpCmd,
    "/analyze": analyzeCmd,
    "/getparams": getParamsCmd,
}

Чтобы в боте отобразить список доступных команд (при нажатии на иконку со слешем "/"), в BotFather необходимо перечислить эти команды. Для этого необходимо отправить команду /setcommands и список команда в таком формате:

start - Начало работы с ботом<br/>
help - Помощь<br/>
analyze - Выполнение анализа<br/>
getparams - Просмотр ранее заданных входных параметров для анализа

In [13]:
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    if message.text in botCmds:
        botCmds[message.text](message)
    else:
        bot.send_message(message.from_user.id, "Я тебя не понимаю. Напиши /help.")

Настраиваем бота на постоянный опрос сервера телеграммма, чтобы узнавать, написал ли кто-то в наш бот

In [ ]:
bot.polling(none_stop=True, interval=0)

Для локального запуска сервера в консоле необходимо запустить скрипт:<br/>
python InvestmentSampleBot.py<br/><br/> 

Или выполнить весь код в Jupyter Notebook. Ячейка с polling'ом будет выполняться до тех пор, пока среда выполнения не будет вручную остановлена.